# Solve a basic optimization model

In this example, we explain the basic functions of the linopy Model class. First, we are setting up a very simple linear optimization model, given by 

Minimize:
$$ x + 2y $$
      
subject to:

$$ x \ge 0 $$
$$y \ge 0 $$
$$3x + 7y \ge 10 $$
$$5x + 2y \ge 3 $$

In [ ]:
from linopy import Model

In [ ]:
m = Model()

The Model class serves at a container for all the relevant data. 

### Adding variables

Let's add the two variables. Note that a variable can always be assigned with a lower and an upper bound. In this case, both `x` and `y` have a lower bound of zero (coming from the first two constraints). Note, the default for lower and upper bounds are minus and plus infinity.

In [ ]:
x = m.add_variables(lower=0, name='x')
y = m.add_variables(lower=0, name='y');

`x` and `y` are linopy variables. Each of them consist of an array with the variable references in the model, that is the names of variables that will finally be used when solving the model. 

In [ ]:
x

Since both `x` and `y` are scalar variables, so their arrays also contain just one variable reference. The variable `x` points to the optimisation variable 1 and the variable `y` points to the optimisation variable 2. Later we will see the benefit of this behaviour.  

### Adding Constraints

Constraints consist of the left hand side (lhs) and the righ hand side (rhs). The lhs must contain all the variables with the corresponding coefficients, while the rhs is just constant. The first lhs of our two remaining constraints is 
$3x + 7y$, which we can write just exactly in this way

In [ ]:
3*x + 7*y

When assigning to the model, we call the function `m.add_constraints`.

In [ ]:
m.add_constraints(3*x + 7*y >= 10)
m.add_constraints(5*x + 2*y >= 3);

## Adding the Objective 

We do the same for defining the objective while the objective function only consists of a linear expression.

In [ ]:
m.add_objective(x + 2*y)

In [ ]:
m.solve()

The solution of the linear problem is stored in `m.solution` in fom of a `xarray.Dataset`. 

In [ ]:
m.solution

## Expanding the dimensionality

Now comes the interesting part. Suppose the two variables `x` and `y` are a function of time `t` and a modified problem setup like 

Minimize:
$$\sum_t x_t + 2 y_t$$

subject to:

$$
x_t \ge 0 \qquad \forall t \\
y_t \ge 0 \qquad \forall t \\
3x_t + 7y_t \ge 10 t \qquad \forall t\\
5x_t + 2y_t \ge 3 t \qquad \forall t
$$

whereas `t` spans all the range from 0 to 10.

First, we define a new model (we just overwrite the old `m`).

In [ ]:
m = Model()

Again, we define `x` and `y` using the `add_variables` function, but now we are adding a `coords` argument. This automatically creates optimization variables for all coordinates, in this case time-steps.

In [ ]:
import pandas as pd
time = pd.Index(range(10), name='time')

x = m.add_variables(lower=0, coords=[time], name='x', )
y = m.add_variables(lower=0, coords=[time], name='y')

We again write the constraints out using the syntax from above, while multiplying the rhs with `t`. Note that the coordinates from the lhs and the rhs have to match. 

*Note: In case lhs and rhs have different sets of coordinates, the constraint creation is broadcasted over all combinations of coordinates.*

In [ ]:
factor = pd.Series(time, index=time)

m.add_constraints(3*x + 7*y >= 10*factor, name='Constraint1')
m.add_constraints(5*x + 2*y >= 3*factor, name='Constraint2')
m

When we add the objective, we use the `sum` function of `linopy.LinearExpression`. This stacks the `term_` dimension, such that all terms of the `time` dimension are rewritten into one big expression. 

In [ ]:
obj = (x + 2*y).sum()
m.add_objective(obj)

In [ ]:
m.solve()

In [ ]:
m.solution.to_dataframe().plot(grid=True, ylabel='Optimal Value')

You can easily add dimensions in this way. Since the underlying code is highly relying on `xarray` broadcasting on new dimensions is automatically supported, so even the `rhs` can introduce new dimensions. Just be careful that all variable dimensions have the correct dimension name. 

We limit the slope of the `y` variable by adding a constraint in the form of 
$$ y_{t} - y_{t-1} \le 0.5 \qquad \forall t \ge 1$$

In [ ]:
lhs = (y - y.shift(time=1)).sel(time=time[1:])
m.add_constraints(lhs, '<=', 0.5, name='Limited growth y')

In [ ]:
m.solve()
m.solution.to_dataframe().plot(grid=True, ylabel='Optimal Value');